#Загрузка данных

In [ ]:
import re
import nltk
import contractions
import numpy as np
import unicodedata as ucd
import spacy
from corus import load_wiki
from nltk.corpus import stopwords
from num2words import num2words
from razdel import tokenize, sentenize
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from spacy import displacy
from gensim.utils import tokenize
from nltk import word_tokenize
from natasha import Doc, Segmenter, NewsEmbedding, NewsMorphTagger

In [ ]:
!pip install corus
!pip install razdel
!pip install contractions
!pip install num2words
!python3 -m spacy download ru_core_news_sm
!pip install natasha
!pip install pymystem3
!wget https://dumps.wikimedia.org/ruwiki/latest/ruwiki-latest-pages-articles.xml.bz2

In [ ]:
np.random.seed(42)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
stop_words = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
path = '/content/ruwiki-latest-pages-articles.xml.bz2'
records = load_wiki(path)
next(records)

WikiRecord(
    id='7',
    url='?curid=7',
    title='Литва',
    text='Литва\n\nЛитва́ ( ), официальное название\xa0— Лито́вская Респу́блика ()\xa0— государство, расположенное в Северной Европе. Площадь\xa0— км². Протяжённость с севера на юг\xa0— 280\xa0км, а с запада на восток\xa0— 370 км. Население составляет человек (август, 2023). Занимает 137-е место в мире по численности населения и 121-е по территории. Имеет выход к Балтийскому морю, расположена на его восточном побережье. Береговая линия составляет всего 99 км (наименьший показатель среди государств Балтии). На севере граничит с Латвией, на юго-востоке\xa0— с Белоруссией, на юго-западе\xa0— с Польшей и Калининградской областью России. По площади и населению является самым крупным государством из стран Балтии.Столица\xa0— Вильнюс. Официальный язык\xa0— литовский. Денежная единица\xa0— евро.Восстановление независимости страны провозглашено 11 марта 1990\xa0года. 6 сентября 1991\xa0года Государственный совет СССР признал независ

In [ ]:
dataset = [next(records).text.lower() for _ in range(1000)]
print(dataset[0])

# Предложения
sentences = [_.text for _ in list(sentenize(dataset[0]))]
print(sentences[0])

россия

росси́я, или росси́йская федера́ция (), — государство в восточной европе и северной азии. россия — крупнейшее государство в мире, её территория в международно признанных границах составляет км². население страны в тех же границах, но включая территорию украинского крыма, аннексия россией которого не получила международного признания, составляет чел. (; 9-е место в мире).столица — москва. государственный язык на всей территории страны — русский, в ряде регионов россии также установлены свои государственные и официальные языки. денежная единица — российский рубль.россия — многонациональное государство с широким этнокультурным многообразием. согласно результатам переписей населения россии 2010 года, а также аннексированных россией украинских крыма и севастополя 2014 года, в стране живут представители свыше 190 национальностей, среди которых русские составляют свыше 80 %, а русским языком владеют свыше 99,4 % россиян. бо́льшая часть населения (около 75 %) в религиозном отношении от

#Удаление стоп-слов

In [ ]:
def remove_stopwords(words):
    """
    Удаление стоп-слов из списка токенизированных слов
    """
    new_words = []

    for word in words:
        if word not in stop_words:
            new_words.append(word)
    return new_words

#Нормализация

In [ ]:
def replace_contractions(text):
    """
    Заменить сокращения в тексте
    """
    return contractions.fix(text)


sample = replace_contractions(dataset[0])
print(sample)
words = nltk.word_tokenize(sample)
print(words)

россия

росси́я, или росси́йская федера́ция (), — государство в восточной европе и северной азии. россия — крупнейшее государство в мире, её территория в международно признанных границах составляет км². население страны в тех же границах, но включая территорию украинского крыма, аннексия россией которого не получила международного признания, составляет чел. (; 9-е место в мире).столица — москва. государственный язык на всей территории страны — русский, в ряде регионов россии также установлены свои государственные и официальные языки. денежная единица — российский рубль.россия — многонациональное государство с широким этнокультурным многообразием. согласно результатам переписей населения россии 2010 года, а также аннексированных россией украинских крыма и севастополя 2014 года, в стране живут представители свыше 190 национальностей, среди которых русские составляют свыше 80 %, а русским языком владеют свыше 99,4 % россиян. бо́льшая часть населения (около 75 %) в религиозном отношении от

In [ ]:
def remove_non_ASCII(words):
    """
    Удаление символов, не являющихся символами ASCII, из списка токенизированных слов
    """
    new_words = []

    for word in words:
      new_word = ucd.normalize('NFKD', word)
      new_words.append(new_word)
    return new_words


def to_lowercase(words):
    """
    Преобразование всех символов в строчные из списка токенизированных слов
    """
    new_words = []

    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """
    Удаление знаков препинания из списка токенизированных слов
    """
    new_words = []

    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """
    Заменить все числа в списке токенизированных слов текстовым представлением
    """
    new_words = []

    for word in words:
        if word.isdigit():
            new_word = num2words(word, lang='ru')
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_eng_words(words):
    """
    Удаление английских символов, слов и небольшая очистка
    """
    substitute = []
    new_words = []
    substitute_1 = re.sub(u"[a-zA-Z]", " ", ' '.join(words))
    substitute_2 = re.sub(r"(\d)+.", "\1", substitute_1)
    substitute_3 = re.sub(r"[\s]\s+", "", substitute_2)
    new_words = re.sub(r"", "", substitute_3)
    return new_words

def normalize(words):
    """
    Нормализация слов
    """
    words = remove_non_ASCII(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    words = remove_eng_words(words)
    return words

words = nltk.word_tokenize(sample)
words = normalize(words)
print(words)

россия россия россииская федерация государство восточнои европе севернои азии россия крупнеишее государство мире территория международно признанных границах составляет кмнаселение страны тех границах включая территорию украинского крыма аннексия россиеи которого получила международного признания составляет чел  место мире столица москва государственныи язык всеи территории страны русскии ряде регионов россии также установлены свои государственные официальные языки денежная единица россиискии рубльроссия многонациональное государство широким этнокультурным многообразием согласно результатам переписеи населения россии две тысячи десять года также аннексированных россиеи украинских крыма севастополя две тысячи четырнадцать года стране живут представители свыше сто девяносто национальностеи среди которых русские составляют свыше восемьдесят русским языком владеют свыше девятьсот девяносто четыре россиян большая часть населения около семьдесят пять религиозном отношении относит православию 

#Лемматизация/Стемминг

In [ ]:
def stem_words(language, words):
    """
    Стемминг в списке токенизированных слов
    """
    stemmer = SnowballStemmer(language)
    stems = []

    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """
    Лемматизировать глаголы в списке токенизированных слов
    """
    lemmatizer = WordNetLemmatizer()
    lemmas = []

    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_and_lemmatize(words):
    stems = stem_words('russian', words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas

last_words = nltk.word_tokenize(words)
stems, lemmas = stem_and_lemmatize(last_words)
print('Исходные лексемы:\n', stems)
print('Результат лемматизации:\n', lemmas)

Исходные лексемы:
 ['росс', 'росс', 'россииск', 'федерац', 'государств', 'восточно', 'европ', 'северно', 'аз', 'росс', 'крупнеиш', 'государств', 'мир', 'территор', 'международн', 'призна', 'границ', 'составля', 'кмнаселен', 'стран', 'тех', 'границ', 'включ', 'территор', 'украинск', 'крым', 'аннекс', 'росс', 'котор', 'получ', 'международн', 'признан', 'составля', 'чел', 'мест', 'мир', 'столиц', 'москв', 'государственны', 'язык', 'все', 'территор', 'стран', 'русск', 'ряд', 'регион', 'росс', 'такж', 'установл', 'сво', 'государствен', 'официальн', 'язык', 'денежн', 'единиц', 'россииск', 'рубльросс', 'многонациональн', 'государств', 'широк', 'этнокультурн', 'многообраз', 'согласн', 'результат', 'перепис', 'населен', 'росс', 'две', 'тысяч', 'деся', 'год', 'такж', 'аннексирова', 'росс', 'украинск', 'крым', 'севастопол', 'две', 'тысяч', 'четырнадца', 'год', 'стран', 'живут', 'представител', 'свыш', 'сто', 'девян', 'национальн', 'сред', 'котор', 'русск', 'составля', 'свыш', 'восемьдес', 'русск'

In [ ]:
sentences = sent_tokenize(words, language='russian')
nlp = spacy.load("ru_core_news_sm")
doc = nlp(words)
cnt = 0

for token in doc:
    if cnt == 10:
        break
    print(token.text, token.dep_, token.head)
    cnt += 1

россия nsubj восточнои
россия appos россия
россииская amod федерация
федерация appos россия
государство appos федерация
восточнои ROOT восточнои
европе iobj восточнои
севернои obj восточнои
азии nmod севернои
россия appos севернои


In [ ]:
displacy.render(doc[2:13], style='dep', jupyter=True)

In [ ]:
# gensim токены
gen_tokens = [word for word in tokenize(words)]
print(f'gensim: {gen_tokens[25:35]}')

# Использование лемматизатора WordNet
wnl = WordNetLemmatizer()
word_net_lemmas = [(word, wnl.lemmatize(word)) for word in gen_tokens]
print(f'\nWordNet: {word_net_lemmas[5:24]}\n')

# Использование Razdel + Natasha
doc = Doc(words)
segmenter = Segmenter()
doc.segment(segmenter)
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
doc.tag_morph(morph_tagger)
print(f'Razdel + Natasha: {doc.tokens[:5]}')
print(doc.sents[:5])

# Использование библиотеки pymystem3
m = Mystem()
lemmas = m.lemmatize(words)
print(f'\npymystem3: {lemmas[5:15]}')
print(m.analyze(words)[5:10])

gensim: ['крыма', 'аннексия', 'россиеи', 'которого', 'получила', 'международного', 'признания', 'составляет', 'чел', 'место']

WordNet: [('восточнои', 'восточнои'), ('европе', 'европе'), ('севернои', 'севернои'), ('азии', 'азии'), ('россия', 'россия'), ('крупнеишее', 'крупнеишее'), ('государство', 'государство'), ('мире', 'мире'), ('территория', 'территория'), ('международно', 'международно'), ('признанных', 'признанных'), ('границах', 'границах'), ('составляет', 'составляет'), ('кмнаселение', 'кмнаселение'), ('страны', 'страны'), ('тех', 'тех'), ('границах', 'границах'), ('включая', 'включая'), ('территорию', 'территорию')]

Razdel + Natasha: [DocToken(stop=6, text='россия', pos='NOUN', feats=<Inan,Nom,Fem,Sing>), DocToken(start=7, stop=13, text='россия', pos='NOUN', feats=<Inan,Nom,Fem,Sing>), DocToken(start=14, stop=24, text='россииская', pos='ADJ', feats=<Nom,Pos,Fem,Sing>), DocToken(start=25, stop=34, text='федерация', pos='NOUN', feats=<Inan,Nom,Fem,Sing>), DocToken(start=35, sto